In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Note: The following do not work with Python 3.12
import shap
# from ydata_profiling import ProfileReport
import sweetviz as sv

Reproducibility:

In [2]:
seed = 2024

# pandas, statsmodels, matplotlib and y_data_profiling rely on numpy's random generator, and thus, we need to set the seed in numpy
np.random.seed(seed)

In [3]:
diet = pd.read_csv('diet.csv', low_memory=False)
diet['Diet'] = diet['Diet'].astype('category')
diet

,AuthorId,Diet,Age
0,10000120E,Vegetarian,46
1,1000014D,Vegan,18
2,1000015A,Vegetarian,58
3,1000016E,Vegetarian,32
4,1000027E,Vegan,61
...,...,...,...
271902,999917E,Vegetarian,28
271903,999936C,Omnivore,22
271904,99993D,Vegetarian,58
271905,99994A,Vegetarian,18


Data Understanding

In [4]:
# user_count= diet['AuthorId']
# print(f'The number of author ids is: {user_count.size}')
# print(f'The number of unique author ids is: {user_count.unique().size}')

Different Graphs of Diet.csv

In [5]:
# # Histogram of Ages
# plt.figure(figsize=(10, 6))
# plt.hist(diet['Age'], bins=20, color='skyblue', edgecolor='black')
# plt.title('Distribution of Ages')
# plt.xlabel('Age')
# plt.ylabel('Frequency')
# plt.show()

In [6]:
# #Box Plot of Ages by Diet: Explore relationships between numerical variables (Age) using a pair plot.
# plt.figure(figsize=(12, 8))
# sns.boxplot(x='Diet', y='Age', data=diet, palette='pastel')
# plt.title('Box Plot of Ages by Diet')
# plt.xlabel('Diet')
# plt.ylabel('Age')
# plt.show()

In [7]:
# #Count Plot of Diets: Show the count of each diet category using a count plot.
# plt.figure(figsize=(10, 6))
# sns.countplot(x='Diet', data=diet, palette='viridis')
# plt.title('Count of Diets')
# plt.xlabel('Diet')
# plt.ylabel('Count')
# plt.show()

In [8]:
requests = pd.read_csv('requests.csv', low_memory=False)
requests['HighProtein'] = requests['HighProtein'].astype('category')
requests['LowSugar'] = requests['LowSugar'].astype('category')
requests

,AuthorId,RecipeId,Time,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
0,2001012259B,73440,1799.950949,0.0,Indifferent,0,0,0
1,437641B,365718,4201.820980,0.0,Yes,0,Indifferent,1
2,1803340263D,141757,6299.861496,0.0,Indifferent,1,Indifferent,0
3,854048B,280351,19801.365796,0.0,Yes,1,0,1
4,2277685E,180505,5400.093457,0.0,Indifferent,0,0,0
...,...,...,...,...,...,...,...,...
140190,163793B,78171,1560.649725,0.0,Indifferent,0,0,1
140191,33888B,333262,1502.011466,1.0,Indifferent,1,0,0
140192,401942C,49200,5999.274269,0.0,Indifferent,0,0,1
140193,346866B,214815,899.523513,0.0,Yes,1,Indifferent,1


In [9]:
# user_count= requests['AuthorId']
# print("Author Id ")
# print(f'The number of author ids is: {user_count.size}')
# print(f'The number of unique author ids is: {user_count.unique().size}')
# print(f'Author Diff: {user_count.size-user_count.unique().size}')
# recipe_count= requests['RecipeId']
# print("Recipe Id ")
# print(f'The number of recipe ids is: {recipe_count.size}')
# print(f'The number of unique recipe ids is: {recipe_count.unique().size}')
# print(f'Recipe Diff: {recipe_count.size-recipe_count.unique().size}')

In [10]:
# #Correlation Heatmap: Visualize the correlation between numerical variables.
# # Exclude non-numeric columns
# numeric_columns = requests[['Time', 'HighCalories', 'LowFat', 'HighFiber']]
# correlation_matrix = numeric_columns.corr()
# 
# # Plot the heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
# plt.title('Correlation Heatmap for Request Variables')
# plt.show()

In [11]:
# #Bar Plot of HighProtein Requests: Visualize the count of HighProtein requests.
# plt.figure(figsize=(8, 5))
# sns.countplot(x='HighProtein', data=requests, palette='Set2')
# plt.title('Count of HighProtein Requests')
# plt.xlabel('HighProtein')
# plt.ylabel('Count')
# plt.show()

In [12]:
# #Bar Plot of LowSugar Requests: Visualize the count of LowSugar requests.
# plt.figure(figsize=(8, 5))
# sns.countplot(x='LowSugar', data=requests, palette='pastel')
# plt.title('Count of LowSugar Requests')
# plt.xlabel('LowSugar')
# plt.ylabel('Count')
# plt.show()

In [13]:
reviews = pd.read_csv('reviews.csv', low_memory=False)
reviews

,AuthorId,RecipeId,Rating,Like,TestSetId
0,2492191A,33671,2.0,NaN,1.0
1,2002019979A,92647,2.0,NaN,2.0
2,408594E,161770,NaN,NaN,3.0
3,2001625557E,108231,2.0,NaN,4.0
4,2001427116E,71109,NaN,NaN,5.0
...,...,...,...,...,...
140190,999595E,338070,2.0,False,NaN
140191,999774A,29002,2.0,False,NaN
140192,999774A,159252,NaN,False,NaN
140193,999774A,1171,2.0,True,NaN


In [14]:
# user1_count= reviews['AuthorId']
# print("Author Id ")
# print(f'The number of author ids is: {user1_count.size}')
# print(f'The number of unique author ids is: {user1_count.unique().size}')
# print(f'Author Diff: {user_count.size-user_count.unique().size}')
# recipe1_count= reviews['RecipeId']
# print("Recipe Id ")
# print(f'The number of recipe ids is: {recipe1_count.size}')
# print(f'The number of unique recipe ids is: {recipe1_count.unique().size}')
# print(f'Recipe Diff: {recipe_count.size-recipe_count.unique().size}')

In [15]:
# #Scatter Plot of Rating vs Like:Investigate the relationship between ratings and likes using a scatter plot
# plt.figure(figsize=(10, 6))
# plt.scatter(reviews['Rating'], reviews['Like'], alpha=0.5, color='green')
# plt.title('Scatter Plot of Rating vs Like')
# plt.xlabel('Rating')
# plt.ylabel('Like')
# plt.show()

In [16]:
recipes = pd.read_csv('recipes.csv', low_memory=False)
recipes.rename(columns={
    'Name': 'RecipeName'
}, inplace=True)
recipes

,RecipeId,RecipeName,CookTime,PrepTime,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield
0,73440,Bow Ties With Broccoli Pesto,0,1800,Other,"c(""\""6\"""", ""\""2\"""", ""\""1 1/2\"""", ""\""1/4\"""", ""\...","c(""\""hazelnuts\"""", ""\""broccoli florets\"""", ""\""...",241.3,10.1,1.2,0.0,13.1,31.8,2.3,1.4,6.7,9.0,NaN
1,365718,Cashew-chutney Rice,3600,600,Other,"c(""\""1\"""", ""\""3/4\"""", ""\""6\"""", ""\""5\"""", ""\""2\""...","c(""\""celery\"""", ""\""onion\"""", ""\""butter\"""", ""\""...",370.8,17.5,7.2,22.9,553.3,44.3,1.6,2.2,9.4,8.0,NaN
2,141757,Copycat Taco Bell Nacho Fries BellGrande,3600,2700,Other,"c(""\""3\"""", ""\""1/2\"""", ""\""1\"""", ""\""1\"""", ""\""3\""...","c(""\""Copycat Taco Bell Seasoned Beef\"""", ""\""ye...",377.6,20.9,10.5,45.7,1501.8,36.6,3.8,6.1,12.9,8.0,NaN
3,280351,Slow Cooker Jalapeno Cheddar Cheese Soup,18000,1800,Other,"c(""\""2\"""", ""\""1\"""", ""\""2\"""", ""\""2\"""", ""\""1\"""",...","c(""\""unsalted butter\"""", ""\""yellow onion\"""", ""...",282.8,16.5,10.3,50.5,630.2,22.8,2.3,2.7,11.7,6.0,NaN
4,180505,Cool & Crisp Citrus Chiffon Pie,3600,1800,Other,"c(""\""1\"""", ""\""1/4\"""", ""\""1/2\"""", ""\""1/2\"""", ""\...","c(""\""unflavored gelatin\"""", ""\""water\"""", ""\""su...",257.5,8.6,2.4,110.7,160.9,39.8,0.4,30.2,6.3,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75599,253577,Frijoles Negros- Crock Pot Mexican Black Beans,43200,28800,Other,"c(""\""2\"""", ""\""6 -8\"""", ""\""5\"""", ""\""1/2\"""", ""\""...","c(""\""black beans\"""", ""\""water\"""", ""\""bay leave...",121.5,0.5,0.1,0.0,1175.1,22.2,7.8,0.6,7.9,NaN,NaN
75600,267827,Moose Moussaka,3600,2700,Other,"c(""\""1\"""", ""\""6\"""", ""\""2\"""", ""\""2\"""", ""\""1/2\""...","c(""\""onion\"""", ""\""garlic cloves\"""", ""\""olive o...",652.2,25.8,10.7,197.9,435.5,51.9,7.5,7.2,50.1,NaN,NaN
75601,266983,Cantonese Pepper Steak for Two (Or More),1800,900,Other,"c(""\""1/2\"""", ""\""1\"""", ""\""1/8\"""", ""\""1/8\"""", ""\...","c(""\""top round steak\"""", ""\""cornstarch\"""", ""\""...",223.9,9.2,3.6,78.3,725.9,7.3,1.1,1.7,26.7,2.0,NaN
75602,253739,Coconut Cream Cooler,300,120,Other,"c(""\""1\"""", ""\""1\"""", ""\""1\"""", ""\""7 1/2\"""", ""\""1...","c(""\""cream of coconut\"""", ""\""water\"""")",2229.8,80.3,69.3,0.0,294.7,369.0,15.7,317.9,26.7,NaN,1 gallon


In [17]:
# recipe1_count= recipes['RecipeId']
# print("Recipe Id ")
# print(f'The number of recipe ids is: {recipe1_count.size}')
# print(f'The number of unique recipe ids is: {recipe1_count.unique().size}')

In [18]:
# #Pair Plot for Nutritional Content:Explore relationships between nutritional content variables using a pair plot.
# nutritional_content = recipes[['Calories', 'FatContent', 'CarbohydrateContent', 'ProteinContent']]
# sns.pairplot(nutritional_content)
# plt.suptitle('Pair Plot of Nutritional Content', y=1.02)
# plt.show()

In [19]:
# #Histogram for Calories:Explore the distribution of calories in recipes using a histogram.
# custom_bins = np.arange(0, 5100, 200)  # Adjust the range and interval as needed
# 
# plt.figure(figsize=(10, 6))
# plt.hist(recipes['Calories'], bins=custom_bins, color='skyblue', edgecolor='black')
# plt.title('Distribution of Calories in Recipes')
# plt.xlabel('Calories')
# plt.ylabel('Frequency')
# 
# # Set x-axis labels
# plt.xticks(custom_bins)
# 
# plt.show()

In [20]:
# #Bar Plot for Recipe Categories: Visualize the distribution of recipes across different categories using a bar plot.
# plt.figure(figsize=(14, 8))
# sns.boxplot(x='RecipeCategory', y='CookTime', data=recipes, palette='Set2')
# plt.title('Box Plot of Cook Time by Recipe Category')
# plt.xlabel('Recipe Category')
# plt.ylabel('Cook Time')
# plt.xticks(rotation=45, ha='right')
# plt.show()


In [21]:
# plt.figure(figsize=(12, 8))
# sns.countplot(x='RecipeCategory', data=recipes, palette='viridis', order=recipes['RecipeCategory'].value_counts().index)
# plt.title('Distribution of Recipes across Categories')
# plt.xlabel('Recipe Category')
# plt.ylabel('Count')
# plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
# plt.show()

In [22]:
merged_request_review = pd.merge(reviews,requests,on=['AuthorId','RecipeId'])
merged_request_review

,AuthorId,RecipeId,Rating,Like,TestSetId,Time,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
0,2492191A,33671,2.0,NaN,1.0,2698.714376,0.0,Indifferent,1,0,1
1,2002019979A,92647,2.0,NaN,2.0,2399.694583,1.0,Indifferent,0,Indifferent,1
2,408594E,161770,NaN,NaN,3.0,2099.113170,1.0,Indifferent,0,0,1
3,2001625557E,108231,2.0,NaN,4.0,1199.645575,1.0,Yes,0,0,1
4,2001427116E,71109,NaN,NaN,5.0,2341.181827,1.0,Indifferent,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
140190,999595E,338070,2.0,False,NaN,3899.421310,0.0,Indifferent,1,Indifferent,0
140191,999774A,29002,2.0,False,NaN,2402.372535,0.0,Indifferent,0,Indifferent,0
140192,999774A,159252,NaN,False,NaN,5999.598903,0.0,Yes,0,0,0
140193,999774A,1171,2.0,True,NaN,480.233207,1.0,Yes,0,0,0


In [23]:
# user1_count= merged_request_review['AuthorId']
# print("Author Id ")
# print(f'The number of author ids is: {user1_count.size}')
# print(f'The number of unique author ids is: {user1_count.unique().size}')
# print(f'Author Diff: {user_count.size-user_count.unique().size}')
# recipe1_count= merged_request_review['RecipeId']
# print("Recipe Id ")
# print(f'The number of recipe ids is: {recipe1_count.size}')
# print(f'The number of unique recipe ids is: {recipe1_count.unique().size}')
# print(f'Recipe Diff: {recipe_count.size-recipe_count.unique().size}')

In [24]:
# #eatmap for Correlation: Visualize the correlation between numerical variables.
# correlation_matrix = merged_request_review[['Rating', 'Like', 'Time', 'HighCalories', 'LowFat', 'HighFiber']].corr()
# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
# plt.title('Correlation Heatmap for Merged Data')
# plt.show()

In [25]:
# plt.figure(figsize=(8, 5))
# sns.barplot(x='HighProtein', y='Rating', data=merged_request_review, palette='Set2')
# plt.title('Average Rating for Recipes with and without High Protein')
# plt.xlabel('HighProtein')
# plt.ylabel('Average Rating')
# plt.show()


In [26]:
# plt.figure(figsize=(10, 6))
# sns.boxplot(x='HighCalories', y='Like', data=merged_request_review, palette='pastel')
# plt.title('Box Plot of Likes for Recipes with and without High Calories')
# plt.xlabel('HighCalories')
# plt.ylabel('Like')
# plt.show() 


In [27]:
# sns.pairplot(merged_request_review[['Like', 'Time', 'HighCalories', 'HighProtein', 'LowFat', 'LowSugar', 'HighFiber']])
# plt.suptitle('Pair Plot of Merged Data', y=1.02)
# plt.show()


**CHECK** If a following recipe id exists and how many times and if the Recipe Id is mapped to the unique AuthorId 

In [28]:
# recipe_id_to_check = merged_request_review['RecipeId'].value_counts().idxmax()
# 
# # Display the result
# recipe_exists = merged_request_review['RecipeId'].isin([recipe_id_to_check])
# 
# # Display the result
# if recipe_exists.any():
#     # print(f"Recipe with RecipeId {recipe_id_to_check} exists.")
#     recipe_count = merged_request_review['RecipeId'].value_counts().get(recipe_id_to_check, 0)
#     print(f"Recipe with RecipeId {recipe_id_to_check} repeats {recipe_count} times.")
#     # Get the table of corresponding AuthorId values for the given RecipeId
#     author_id_table = merged_request_review.loc[merged_request_review['RecipeId'] == recipe_id_to_check, ['AuthorId','RecipeId']]
#     print("Table of AuthorId values for RecipeId", recipe_id_to_check)
#     print()
#     
# else:
#     print(f"Recipe with RecipeId {recipe_id_to_check} does not exist.")
# 
# author_id_table    

In [29]:
# unique_recipe_ids_from_merge = merged_request_review['RecipeId'].unique()
# unique_recipe_ids_from_recipes = recipes['RecipeId'].unique()
# are_ids_equal = set(unique_recipe_ids_from_recipes) == set(unique_recipe_ids_from_merge)
# 
# if are_ids_equal:
#     print("The unique RecipeId values are the same in both DataFrames.")
# else:
#     print("The unique RecipeId values are not the same in both DataFrames.")
# 
# # # Assuming merged_request_review is the result of merging reviews and requests DataFrames
# # # using pd.merge(reviews, requests, on=['AuthorId', 'RecipeId'])
# # 
# # # Get the unique RecipeId values
# # unique_recipe_ids = merged_request_review['RecipeId'].unique()
# # 
# # # Initialize a list to store the uniqueness result for each RecipeId
# # uniqueness_results = []
# # 
# # # Iterate over unique RecipeId values
# # for recipe_id in unique_recipe_ids:
# #     # Get the table of corresponding AuthorId values for the given RecipeId
# #     author_id_table = merged_request_review.loc[merged_request_review['RecipeId'] == recipe_id, ['AuthorId', 'RecipeId']]
# # 
# #     # Check if the mapped AuthorId values are unique
# #     is_author_id_unique = ~author_id_table['AuthorId'].duplicated().any()
# # 
# #     uniqueness_results.append((recipe_id, is_author_id_unique))
# # 
# # # Create a DataFrame from the results
# # uniqueness_df = pd.DataFrame(uniqueness_results, columns=['RecipeId', 'IsAuthorIdUnique'])
# # 
# # # Display the DataFrame
# # print(uniqueness_df)
# # # 
# # # Visualization: Bar plot of unique AuthorId counts for each RecipeId
# # uniqueness_df.set_index('RecipeId')['IsAuthorIdUnique'].astype(int).plot(kind='bar', xlabel='RecipeId', ylabel='IsAuthorIdUnique (1: True, 0: False)', title='Uniqueness of AuthorId for Each RecipeId')
# # plt.show()

In [30]:
merged_recipes_req_review= pd.merge(merged_request_review,recipes,on=['RecipeId'],how='right')
merged_recipes_req_review

,AuthorId,RecipeId,Rating,Like,TestSetId,Time,HighCalories,HighProtein,LowFat,LowSugar,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield
0,914496A,73440,NaN,NaN,1548.0,1800.709345,0.0,Indifferent,0,0,...,10.1,1.2,0.0,13.1,31.8,2.3,1.4,6.7,9.0,NaN
1,2668181C,73440,2.0,NaN,2069.0,1797.825626,0.0,Yes,0,Indifferent,...,10.1,1.2,0.0,13.1,31.8,2.3,1.4,6.7,9.0,NaN
2,2000237272A,73440,2.0,NaN,2152.0,1800.440504,1.0,Indifferent,0,0,...,10.1,1.2,0.0,13.1,31.8,2.3,1.4,6.7,9.0,NaN
3,2001895110E,73440,NaN,NaN,3293.0,1799.437007,0.0,Indifferent,0,0,...,10.1,1.2,0.0,13.1,31.8,2.3,1.4,6.7,9.0,NaN
4,1800347124D,73440,NaN,NaN,4621.0,1801.007172,1.0,Yes,0,0,...,10.1,1.2,0.0,13.1,31.8,2.3,1.4,6.7,9.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140190,1677720D,253577,2.0,True,NaN,71998.977353,0.0,Indifferent,1,Indifferent,...,0.5,0.1,0.0,1175.1,22.2,7.8,0.6,7.9,NaN,NaN
140191,853126B,267827,NaN,NaN,7148.0,6299.278484,0.0,Yes,1,0,...,25.8,10.7,197.9,435.5,51.9,7.5,7.2,50.1,NaN,NaN
140192,2000154789A,266983,2.0,True,NaN,2700.812133,0.0,Yes,0,0,...,9.2,3.6,78.3,725.9,7.3,1.1,1.7,26.7,2.0,NaN
140193,499207A,253739,2.0,True,NaN,419.558713,1.0,Indifferent,0,Indifferent,...,80.3,69.3,0.0,294.7,369.0,15.7,317.9,26.7,NaN,1 gallon


Changing the object data types

Data Joining using common attributes

In [31]:
author_ID = 'AuthorId'
merged_diet_all = pd.merge(diet, merged_recipes_req_review, on=author_ID)
# merged_request_recipes = pd.merge(requests, recipes, on='RecipeId', how='left')
merged_diet_all

,AuthorId,Diet,Age,RecipeId,Rating,Like,TestSetId,Time,HighCalories,HighProtein,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield
0,1000036C,Vegetarian,50,320576,NaN,False,NaN,119.024930,0.0,Indifferent,...,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,1.0,NaN
1,1000216B,Vegetarian,78,189335,NaN,False,NaN,1199.386790,0.0,Yes,...,19.2,4.5,33.9,714.2,38.0,0.8,4.3,16.3,8.0,NaN
2,1000221A,Vegetarian,25,17444,NaN,False,NaN,3899.147999,1.0,Yes,...,10.3,6.0,36.6,119.8,13.7,1.0,1.0,8.7,12.0,NaN
3,1000221A,Vegetarian,25,133043,2.0,False,NaN,362.152341,0.0,Yes,...,13.5,1.9,0.0,0.6,1.9,0.4,0.4,0.2,NaN,4 ounces
4,1000221A,Vegetarian,25,90537,2.0,False,NaN,1198.957497,0.0,Yes,...,49.0,7.2,0.0,115.8,241.7,69.4,15.7,68.9,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140190,999774A,Vegetarian,57,29002,2.0,False,NaN,2402.372535,0.0,Indifferent,...,33.3,8.5,33.2,318.2,107.7,10.3,13.5,32.0,4.0,NaN
140191,999774A,Vegetarian,57,1171,2.0,True,NaN,480.233207,1.0,Yes,...,19.8,12.2,68.3,247.4,87.5,4.0,57.8,18.2,1.0,2 mugs
140192,999774A,Vegetarian,57,159252,NaN,False,NaN,5999.598903,0.0,Yes,...,0.6,0.1,0.0,6.8,240.4,7.0,229.8,1.1,NaN,4 cups
140193,999917E,Vegetarian,28,169413,2.0,False,NaN,3600.387748,0.0,Indifferent,...,8.6,4.7,66.5,521.9,24.8,1.7,3.4,5.6,8.0,NaN


Plot graphs of the data frames

Impute the missing values

In [32]:

# 1 value missing in diet column. Filled with most occuring value.
merged_diet_all['Diet'] = merged_diet_all['Diet'].fillna('Vegetarian')

In [33]:
#impute the values for all dietary preferences for all ages with the most frequent RecipeId for that age in that category
helper_df = merged_diet_all.groupby(['Age', 'Diet'])['RecipeId'].agg(lambda x: x.mode()[0]).reset_index()
helper_df.columns = ['Age', 'Diet', 'Most Common Recipe']
def impute_recipe(row):
    if pd.isnull(row['RecipeId']):
        return helper_df[(helper_df['Age'] == row['Age']) & (helper_df['Diet'] == row['Diet'])]['Most Common Recipe'].values[0]
    else:
        return row['RecipeId']
merged_diet_all['RecipeId'] = merged_diet_all.apply(impute_recipe, axis=1)
merged_diet_all

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


,AuthorId,Diet,Age,RecipeId,Rating,Like,TestSetId,Time,HighCalories,HighProtein,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield
0,1000036C,Vegetarian,50,320576,NaN,False,NaN,119.024930,0.0,Indifferent,...,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,1.0,NaN
1,1000216B,Vegetarian,78,189335,NaN,False,NaN,1199.386790,0.0,Yes,...,19.2,4.5,33.9,714.2,38.0,0.8,4.3,16.3,8.0,NaN
2,1000221A,Vegetarian,25,17444,NaN,False,NaN,3899.147999,1.0,Yes,...,10.3,6.0,36.6,119.8,13.7,1.0,1.0,8.7,12.0,NaN
3,1000221A,Vegetarian,25,133043,2.0,False,NaN,362.152341,0.0,Yes,...,13.5,1.9,0.0,0.6,1.9,0.4,0.4,0.2,NaN,4 ounces
4,1000221A,Vegetarian,25,90537,2.0,False,NaN,1198.957497,0.0,Yes,...,49.0,7.2,0.0,115.8,241.7,69.4,15.7,68.9,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140190,999774A,Vegetarian,57,29002,2.0,False,NaN,2402.372535,0.0,Indifferent,...,33.3,8.5,33.2,318.2,107.7,10.3,13.5,32.0,4.0,NaN
140191,999774A,Vegetarian,57,1171,2.0,True,NaN,480.233207,1.0,Yes,...,19.8,12.2,68.3,247.4,87.5,4.0,57.8,18.2,1.0,2 mugs
140192,999774A,Vegetarian,57,159252,NaN,False,NaN,5999.598903,0.0,Yes,...,0.6,0.1,0.0,6.8,240.4,7.0,229.8,1.1,NaN,4 cups
140193,999917E,Vegetarian,28,169413,2.0,False,NaN,3600.387748,0.0,Indifferent,...,8.6,4.7,66.5,521.9,24.8,1.7,3.4,5.6,8.0,NaN


In [34]:
# Fill the rest of the missing values in the merged_diet_all by mapping them from requests.csv with RecipeId as key 
# Create mapping DataFrames from `requests`
map_time = requests.set_index('RecipeId')['Time'].to_dict()
map_calories = requests.set_index('RecipeId')['HighCalories'].to_dict()
map_protein = requests.set_index('RecipeId')['HighProtein'].to_dict()
map_fat = requests.set_index('RecipeId')['LowFat'].to_dict()
map_sugar = requests.set_index('RecipeId')['LowSugar'].to_dict()
map_fiber = requests.set_index('RecipeId')['HighFiber'].to_dict()

# Apply mapping to `merged_diet_all`
merged_diet_all['Time'] = merged_diet_all['RecipeId'].map(map_time)
merged_diet_all['HighCalories'] = merged_diet_all['RecipeId'].map(map_calories)
merged_diet_all['HighProtein'] = merged_diet_all['RecipeId'].map(map_protein)
merged_diet_all['LowFat'] = merged_diet_all['RecipeId'].map(map_fat)
merged_diet_all['LowSugar'] = merged_diet_all['RecipeId'].map(map_sugar)
merged_diet_all['HighFiber'] = merged_diet_all['RecipeId'].map(map_fiber)
merged_diet_all    

,AuthorId,Diet,Age,RecipeId,Rating,Like,TestSetId,Time,HighCalories,HighProtein,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield
0,1000036C,Vegetarian,50,320576,NaN,False,NaN,119.024930,0.0,Indifferent,...,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,1.0,NaN
1,1000216B,Vegetarian,78,189335,NaN,False,NaN,1199.549763,1.0,Indifferent,...,19.2,4.5,33.9,714.2,38.0,0.8,4.3,16.3,8.0,NaN
2,1000221A,Vegetarian,25,17444,NaN,False,NaN,3899.147999,1.0,Yes,...,10.3,6.0,36.6,119.8,13.7,1.0,1.0,8.7,12.0,NaN
3,1000221A,Vegetarian,25,133043,2.0,False,NaN,362.152341,0.0,Yes,...,13.5,1.9,0.0,0.6,1.9,0.4,0.4,0.2,NaN,4 ounces
4,1000221A,Vegetarian,25,90537,2.0,False,NaN,1198.957497,0.0,Yes,...,49.0,7.2,0.0,115.8,241.7,69.4,15.7,68.9,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140190,999774A,Vegetarian,57,29002,2.0,False,NaN,2400.009731,1.0,Indifferent,...,33.3,8.5,33.2,318.2,107.7,10.3,13.5,32.0,4.0,NaN
140191,999774A,Vegetarian,57,1171,2.0,True,NaN,480.233207,1.0,Yes,...,19.8,12.2,68.3,247.4,87.5,4.0,57.8,18.2,1.0,2 mugs
140192,999774A,Vegetarian,57,159252,NaN,False,NaN,5998.670408,1.0,Indifferent,...,0.6,0.1,0.0,6.8,240.4,7.0,229.8,1.1,NaN,4 cups
140193,999917E,Vegetarian,28,169413,2.0,False,NaN,3600.387748,0.0,Indifferent,...,8.6,4.7,66.5,521.9,24.8,1.7,3.4,5.6,8.0,NaN


Impute value from recipe.csv

In [35]:
map_name = recipes.set_index('RecipeId')['RecipeName'].to_dict()
map_cook_time = recipes.set_index('RecipeId')['CookTime'].to_dict()
map_prep_time = recipes.set_index('RecipeId')['PrepTime'].to_dict()
map_category = recipes.set_index('RecipeId')['RecipeCategory'].to_dict()
map_quantities = recipes.set_index('RecipeId')['RecipeIngredientQuantities'].to_dict()
map_parts = recipes.set_index('RecipeId')['RecipeIngredientParts'].to_dict()
map_calories = recipes.set_index('RecipeId')['Calories'].to_dict()
map_fat_content = recipes.set_index('RecipeId')['FatContent'].to_dict()
map_saturated_content = recipes.set_index('RecipeId')['SaturatedFatContent'].to_dict()
map_cholesterol = recipes.set_index('RecipeId')['CholesterolContent'].to_dict()
map_sodium = recipes.set_index('RecipeId')['SodiumContent'].to_dict()
map_carbohydrate = recipes.set_index('RecipeId')['CarbohydrateContent'].to_dict()
map_fiber = recipes.set_index('RecipeId')['FiberContent'].to_dict()
map_sugar = recipes.set_index('RecipeId')['SugarContent'].to_dict()
map_protein = recipes.set_index('RecipeId')['ProteinContent'].to_dict()
map_servings = recipes.set_index('RecipeId')['RecipeServings'].to_dict()
map_yield = recipes.set_index('RecipeId')['RecipeYield'].to_dict()


# Apply mapping to `merged_diet_all`
merged_diet_all['RecipeName'] = merged_diet_all['RecipeId'].map(map_name)
merged_diet_all['CookTime'] = merged_diet_all['RecipeId'].map(map_cook_time)
merged_diet_all['PrepTime'] = merged_diet_all['RecipeId'].map(map_prep_time)
merged_diet_all['RecipeCategory'] = merged_diet_all['RecipeId'].map(map_category)
merged_diet_all['RecipeIngredientQuantities'] = merged_diet_all['RecipeId'].map(map_quantities)
merged_diet_all['RecipeIngredientParts'] = merged_diet_all['RecipeId'].map(map_parts)
merged_diet_all['Calories'] = merged_diet_all['RecipeId'].map(map_calories)
merged_diet_all['FatContent'] = merged_diet_all['RecipeId'].map(map_fat)
merged_diet_all['SaturatedFatContent'] = merged_diet_all['RecipeId'].map(map_saturated_content)
merged_diet_all['CholesterolContent'] = merged_diet_all['RecipeId'].map(map_cholesterol)
merged_diet_all['SodiumContent'] = merged_diet_all['RecipeId'].map(map_sodium)
merged_diet_all['CarbohydrateContent'] = merged_diet_all['RecipeId'].map(map_carbohydrate)
merged_diet_all['FiberContent'] = merged_diet_all['RecipeId'].map(map_fiber)
merged_diet_all['SugarContent'] = merged_diet_all['RecipeId'].map(map_sugar)
merged_diet_all['ProteinContent'] = merged_diet_all['RecipeId'].map(map_protein)
merged_diet_all['RecipeServings'] = merged_diet_all['RecipeId'].map(map_servings)
merged_diet_all['RecipeYield'] = merged_diet_all['RecipeId'].map(map_yield)

**CHECK IF IT WORKS**

In [74]:
merged_diet_all['Rating'] = merged_diet_all['Rating'].fillna(0)
merged_diet_all['TestSetId'] = merged_diet_all['TestSetId']
merged_diet_all

,AuthorId,Diet,Age,RecipeId,Rating,Like,TestSetId,Time,HighCalories,HighProtein,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield
0,1000036C,Vegetarian,50,320576,0.0,False,NaN,119.024930,0.0,Indifferent,...,0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,1.0,NaN
1,1000216B,Vegetarian,78,189335,0.0,False,NaN,1199.549763,1.0,Indifferent,...,0,4.5,33.9,714.2,38.0,0.8,4.3,16.3,8.0,NaN
2,1000221A,Vegetarian,25,17444,0.0,False,NaN,3899.147999,1.0,Yes,...,0,6.0,36.6,119.8,13.7,1.0,1.0,8.7,12.0,NaN
3,1000221A,Vegetarian,25,133043,2.0,False,NaN,362.152341,0.0,Yes,...,0,1.9,0.0,0.6,1.9,0.4,0.4,0.2,NaN,4 ounces
4,1000221A,Vegetarian,25,90537,2.0,False,NaN,1198.957497,0.0,Yes,...,0,7.2,0.0,115.8,241.7,69.4,15.7,68.9,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140190,999774A,Vegetarian,57,29002,2.0,False,NaN,2400.009731,1.0,Indifferent,...,0,8.5,33.2,318.2,107.7,10.3,13.5,32.0,4.0,NaN
140191,999774A,Vegetarian,57,1171,2.0,True,NaN,480.233207,1.0,Yes,...,0,12.2,68.3,247.4,87.5,4.0,57.8,18.2,1.0,2 mugs
140192,999774A,Vegetarian,57,159252,0.0,False,NaN,5998.670408,1.0,Indifferent,...,0,0.1,0.0,6.8,240.4,7.0,229.8,1.1,NaN,4 cups
140193,999917E,Vegetarian,28,169413,2.0,False,NaN,3600.387748,0.0,Indifferent,...,0,4.7,66.5,521.9,24.8,1.7,3.4,5.6,8.0,NaN


In [91]:
# Assuming df is your DataFrame
df = merged_diet_all
# Calculate the proportion of true and false values in the 'Like' column
import pandas as pd
import numpy as np

# Assuming df is your DataFrame
# Calculate the proportion of true and false values in the 'Like' column
proportion_true = df['Like'].mean()
proportion_false = 1 - proportion_true

# Identify the indices of false values
false_indices = df[df['Like'] == False].index

# Randomly sample false indices to achieve a balanced proportion
num_false_to_sample = int(df['Like'].value_counts()[True] / proportion_true) - df['Like'].value_counts()[False]
indices_to_sample = np.random.choice(false_indices, size=num_false_to_sample, replace=False)

# Create a new DataFrame with sampled false values
df_balanced = pd.concat([df[df['Like'] == True], df.loc[indices_to_sample]])

# Shuffle the new DataFrame to randomize the order
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Print the proportion of true and false values in the balanced DataFrame
print(df_balanced['Like'].value_counts(normalize=True))


Like
True     0.5
False    0.5
Name: proportion, dtype: float64


In [92]:
df_balanced

,AuthorId,Diet,Age,RecipeId,Rating,Like,TestSetId,Time,HighCalories,HighProtein,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield
0,2000505765C,Vegetarian,44,439034,0.0,True,NaN,1199.936304,0.0,Indifferent,...,0,0.7,61.6,579.6,14.0,1.4,3.1,19.6,4.0,NaN
1,2419620E,Vegan,31,208619,0.0,True,NaN,1199.524490,1.0,Yes,...,0,27.1,29.4,134.2,112.5,8.6,76.5,18.5,NaN,NaN
2,2655564E,Vegetarian,25,45557,0.0,False,NaN,2699.697526,1.0,Indifferent,...,0,4.5,53.4,294.0,43.9,0.6,32.0,3.3,18.0,NaN
3,2000324201C,Vegetarian,62,1481,2.0,True,NaN,4801.663258,0.0,Indifferent,...,1,0.4,0.0,3.9,8.3,1.6,4.2,1.3,NaN,1 platter
4,2002752875E,Vegetarian,54,178401,0.0,False,NaN,10800.160212,0.0,Indifferent,...,0,4.7,92.8,702.3,28.5,0.5,22.5,30.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25751,1803316815C,Vegetarian,56,209347,0.0,False,NaN,4200.410191,1.0,Indifferent,...,1,0.4,65.8,75.7,1.0,0.0,0.3,26.2,4.0,NaN
25752,2001055509B,Vegetarian,66,18510,2.0,True,NaN,119.221052,0.0,Yes,...,0,9.9,216.5,439.8,9.0,7.7,0.6,11.5,1.0,1 1 Bun or 2 slices
25753,1184634D,Vegetarian,68,43522,0.0,True,NaN,3599.134079,1.0,Indifferent,...,0,25.8,194.3,668.9,102.7,3.3,60.7,11.3,6.0,1 cake
25754,2000760057B,Vegetarian,65,193732,0.0,False,NaN,6301.568932,0.0,Yes,...,0,7.8,36.7,297.3,3.6,0.4,1.2,7.7,NaN,NaN


In [37]:
# final_data = merged_diet_all
# final_data['AuthorId'], unique_authorids = pd.factorize(final_data['AuthorId'])
# 
# final_data

In [38]:
# diet_mapping = {'Vegan': 1, 'Vegetarian': 2, 'Omnivore': 3}
# protien_mapping = {'Yes': 1, 'Indifferent': 0}
# sugar_mapping = { 'Indifferent': 1,'0':0}
# # RECIPE CATEGORY TRAINNNNNNNNNNNNN
# 
# # Apply mapping to the 'Diet' column
# merged_diet_all['Diet'] = merged_diet_all['Diet'].map(diet_mapping)
# merged_diet_all['HighProtein'] = merged_diet_all['HighProtein'].map(protien_mapping)
# merged_diet_all['LowSugar'] = merged_diet_all['LowSugar'].map(sugar_mapping)
# 
# merged_diet_all

,AuthorId,Diet,Age,RecipeId,Rating,Like,TestSetId,Time,HighCalories,HighProtein,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield
0,1000036C,2,50,320576,0.0,False,NaN,119.024930,0.0,0,...,0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,1.0,NaN
1,1000216B,2,78,189335,0.0,False,NaN,1199.549763,1.0,0,...,0,4.5,33.9,714.2,38.0,0.8,4.3,16.3,8.0,NaN
2,1000221A,2,25,17444,0.0,False,NaN,3899.147999,1.0,1,...,0,6.0,36.6,119.8,13.7,1.0,1.0,8.7,12.0,NaN
3,1000221A,2,25,133043,2.0,False,NaN,362.152341,0.0,1,...,0,1.9,0.0,0.6,1.9,0.4,0.4,0.2,NaN,4 ounces
4,1000221A,2,25,90537,2.0,False,NaN,1198.957497,0.0,1,...,0,7.2,0.0,115.8,241.7,69.4,15.7,68.9,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140190,999774A,2,57,29002,2.0,False,NaN,2400.009731,1.0,0,...,0,8.5,33.2,318.2,107.7,10.3,13.5,32.0,4.0,NaN
140191,999774A,2,57,1171,2.0,True,NaN,480.233207,1.0,1,...,0,12.2,68.3,247.4,87.5,4.0,57.8,18.2,1.0,2 mugs
140192,999774A,2,57,159252,0.0,False,NaN,5998.670408,1.0,0,...,0,0.1,0.0,6.8,240.4,7.0,229.8,1.1,NaN,4 cups
140193,999917E,2,28,169413,2.0,False,NaN,3600.387748,0.0,0,...,0,4.7,66.5,521.9,24.8,1.7,3.4,5.6,8.0,NaN


In [39]:
# # Function to clean the the strings
# import pandas as pd
# 
# # Updated clean_string function
# def clean_string(string):
#     # Remove 'c(' and ')' and replace escaped backslashes and double quotes
#     cleaned_string = string.replace('c(', '').replace(')', '').replace('\\', '').replace('\"', '')
# 
#     # Split the cleaned string by commas and remove extra spaces
#     parsed_string = [val.strip() for val in cleaned_string.split(',')]
# 
#     # Convert string elements to int
#     parsed_string = [int(val) if val.isdigit() else val for val in parsed_string]
# 
#     return parsed_string
# 
# # Read the CSV file into a DataFrame
# df = pd.read_csv('recipes.csv')
# 
# # Apply the updated clean_string function to the RecipeIngredientQuantities column
# df['RecipeIngredientQuantities'] = df['RecipeIngredientQuantities'].apply(clean_string)
# df['RecipeIngredientParts'] = df['RecipeIngredientParts'].apply(clean_string)
# 
# #Display the DataFrame with updated and cleaned values
# df

In [40]:
# # Function to calculate RecipeServings based on quantities
# def calculate_servings(quantities):
#     try:
#         # Sum up the quantities to get the total amount of ingredients
#         total_quantity = sum(eval(quantities))
# 
#         # Assuming a base serving size, you can adjust this based on your data
#         base_serving_size = 1
# 
#         # Calculate the servings based on the total quantity and the base serving size
#         servings = total_quantity / base_serving_size
#         return servings
#     except (ValueError, SyntaxError) as e:
#         # Handle the case where the calculation fails
#         print(f"Error calculating servings: {e}")
#         return None
# 
# # Apply the function to calculate RecipeServings
# df['RecipeServings'] = df['RecipeIngredientQuantities'].apply(calculate_servings)
# df

Export data frame to csv file to train the model

In [50]:
columns_to_drop = ['RecipeIngredientQuantities', 'RecipeIngredientParts', 'RecipeServings', 'RecipeYield', 'RecipeName', 'RecipeCategory']
merged_diet_all = merged_diet_all.drop(columns_to_drop, axis=1)

# final_data.to_csv('final_data.csv', index=False)
merged_diet_all

KeyError: "['RecipeIngredientQuantities', 'RecipeIngredientParts', 'RecipeServings', 'RecipeYield', 'RecipeName', 'RecipeCategory'] not found in axis"

In [ ]:
# df = final_data.loc[final_data['CookTime'] <= 7200]
# df
# final_data.drop(final_data[final_data['CookTime'] > 7200].index, inplace=True)
# final_data

Random Forest Classifier

In [93]:
# final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 312123 entries, 0 to 362752
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   Diet                 312123 non-null  category
 1   Age                  312123 non-null  int64   
 2   Like                 90758 non-null   object  
 3   TestSetId            312123 non-null  float64 
 4   HighCalories         312123 non-null  float64 
 5   HighProtein          312123 non-null  int64   
 6   LowFat               312123 non-null  int64   
 7   LowSugar             312123 non-null  int64   
 8   HighFiber            312123 non-null  int64   
 9   CookTime             312123 non-null  int64   
 10  Calories             312123 non-null  float64 
 11  FatContent           312123 non-null  int64   
 12  SaturatedFatContent  312123 non-null  float64 
 13  CholesterolContent   312123 non-null  float64 
 14  SodiumContent        312123 non-null  float64 
 15  Carbo

In [42]:
# df = merged_diet_all.copy()
# 
# 
# 
# # Drop rows with NaN values in the target column ('Like') for training set
# train_data = df[df['TestSetId'].isna()]
# train_data = train_data.drop('TestSetId', axis=1)
# train_data

,AuthorId,Diet,Age,RecipeId,Rating,Like,Time,HighCalories,HighProtein,LowFat,...,PrepTime,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
0,1000036C,2,50,320576,0.0,False,119.024930,0.0,0,0,...,120,138.7,0,0.0,0.0,0.6,0.0,0.0,0.0,0.0
1,1000216B,2,78,189335,0.0,False,1199.549763,1.0,0,0,...,600,395.7,0,4.5,33.9,714.2,38.0,0.8,4.3,16.3
2,1000221A,2,25,17444,0.0,False,3899.147999,1.0,1,0,...,2100,181.7,0,6.0,36.6,119.8,13.7,1.0,1.0,8.7
3,1000221A,2,25,133043,2.0,False,362.152341,0.0,1,0,...,300,125.6,0,1.9,0.0,0.6,1.9,0.4,0.4,0.2
4,1000221A,2,25,90537,2.0,False,1198.957497,0.0,1,0,...,1200,1590.1,0,7.2,0.0,115.8,241.7,69.4,15.7,68.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140189,999595E,2,31,338070,2.0,False,3899.421310,0.0,0,1,...,900,464.3,1,15.0,99.8,391.7,43.1,1.4,27.3,5.2
140190,999774A,2,57,29002,2.0,False,2400.009731,1.0,0,0,...,1200,838.5,0,8.5,33.2,318.2,107.7,10.3,13.5,32.0
140191,999774A,2,57,1171,2.0,True,480.233207,1.0,1,0,...,120,665.9,0,12.2,68.3,247.4,87.5,4.0,57.8,18.2
140192,999774A,2,57,159252,0.0,False,5998.670408,1.0,0,0,...,1200,928.0,0,0.1,0.0,6.8,240.4,7.0,229.8,1.1


In [95]:
from sklearn.model_selection import train_test_split

# Assuming 'final_data' has features and the target variable 'Like'

# Drop rows with NaN values in the target column ('Like') for training set
train_data = df_balanced.dropna(subset=['Like'])

# Assuming 'X' contains features and 'y' is the target variable
X = train_data.drop(columns=['Like'])
y = train_data['Like']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=3)


In [116]:
### data wrangling

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


transform_scaler = StandardScaler()

# dimensionality reduction
transform_pca = PCA()

# value imputing

# outlier detection/removal

In [117]:
# # split data into learning and test sets
# from sklearn.model_selection import train_test_split
# 
# X_train, X_test, y_train, y_test = \
#     train_test_split(final_data.iloc[:, :-1], train_data.iloc[:, -1:],
#                      test_size=0.3,
#                      shuffle=True,
#                      random_state=3)
from sklearn.model_selection import train_test_split

# Assuming 'final_data' has features and the target variable 'Like'

# Drop rows with NaN values in the target column ('Like') for training set
train_data = final_data.dropna(subset=['Like'])

# Assuming 'X' contains features and 'y' is the target variable
X = train_data.drop(columns=['Like'])
y = train_data['Like'].astype(bool)


# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=3)
y_train.info()

<class 'pandas.core.series.Series'>
Index: 63530 entries, 116252 to 287567
Series name: Like
Non-Null Count  Dtype
--------------  -----
63530 non-null  bool 
dtypes: bool(1)
memory usage: 558.4 KB


In [ ]:
# Here, you want to find the best classifier. As candidates, consider
#   1. LogisticRegression
#   2. RandomForestClassifier
#   3. other algorithms from sklearn (easy to add)
#   4. custom algorithms (more difficult to implement)
# 
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# 
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import GridSearchCV
# 
# model_logistic_regression = LogisticRegression(max_iter=30)
# model_random_forest = RandomForestClassifier()
# model_gradient_boosting = GradientBoostingClassifier()
# 
# # train the models
# pipeline = Pipeline(steps=[("scaler", transform_scaler),
#                            ("pca", transform_pca),
#                            ("model", None)])
# 
# parameter_grid_preprocessing = {
#     "pca__n_components" : [1, 2, 3, 4],
# }
# 
# parameter_grid_logistic_regression = {
#     "model" : [model_logistic_regression],
#     "model__C" : [0.1, 1, 10],  # inverse regularization strength
# }
# 
# parameter_grid_gradient_boosting = {
#     "model" : [model_gradient_boosting],
#     "model__n_estimators" : [10, 20, 30]
# }
# 
# parameter_grid_random_forest = {
#     "model" : [model_random_forest],
#     "model__n_estimators" : [10, 20, 50],  # number of max trees in the forest
#     "model__max_depth" : [2, 3, 4],
# }
# 
# meta_parameter_grid = [parameter_grid_logistic_regression,
#                        parameter_grid_random_forest,
#                        parameter_grid_gradient_boosting]
# 
# meta_parameter_grid = [{**parameter_grid_preprocessing, **model_grid}
#                        for model_grid in meta_parameter_grid]
# 
# search = GridSearchCV(pipeline,
#                       meta_parameter_grid,
#                       scoring="balanced_accuracy",
#                       n_jobs=2,
#                       cv=5,  # number of folds for cross-validation 
#                       error_score="raise"
#                       )
# 
# # here, the actual training and grid search happens
# search.fit(X_train, y_train.values.ravel())
# 
# print("best parameter:", search.best_params_ ,"(CV score=%0.3f)" % search.best_score_)

In [99]:
# # evaluate performance of model on test set
# print("Score on test set:", search.score(X_test, y_test.values.ravel()))
# 
# # contingency table
# ct = pd.crosstab(search.best_estimator_.predict(X_test), y_test.values.ravel(),
#                  rownames=["pred"], colnames=["true"])
# print(ct)

NameError: name 'search' is not defined

In [ ]:
# (optional, if you're curious) 
# for a detailed look on the performance of the different models
# def get_search_score_overview():
#     for c,s in zip(search.cv_results_["params"],search.cv_results_["mean_test_score"]):
#         print(c, s)
# 
# print(get_search_score_overview())

In [ ]:
# assume random forest model
# model = RandomForestClassifier(n_estimators=10, random_state=seed)
# model.fit(X_train, y_train.values.ravel())
# 
# # compute shapley values
# explainer = shap.TreeExplainer(model)
# shap_values = explainer.shap_values(X_train)
# shap_interaction_values = explainer.shap_interaction_values(X_train)
# 
# expected_value = explainer.expected_value
# print(expected_value)

In [ ]:
# # class dependent plots of shapley values for each feature
# for i,c in enumerate(final_data.variety.unique()):
#     shap.summary_plot(shap_values[i], X_train, show=False)
#     plt.title("Shapley values for "+str(c))
#     plt.show()

In [ ]:
# def micro_service_classify_iris(datapoint):
# 
#     # make sure the provided datapoints adhere to the correct format for model input
# 
#     # fetch your trained model
#     model = search.best_estimator_
# 
#     # make prediction with the model
#     prediction = model.predict(datapoint)
# 
#     return prediction


In [ ]:
# # hypothetical new batch of flowers arrives
# from scipy.stats import norm
# 
# amount_of_new_flowers = 9
# df_flowers = pd.DataFrame(columns=df.columns.drop("variety"), index=range(1, amount_of_new_flowers+1))
# 
# for i in df_flowers.index:
#     df_flowers.loc[i, "sepal.length"] = norm(loc=6, scale=2).rvs()
#     df_flowers.loc[i, "sepal.width"] = norm(loc=3, scale=1).rvs()
#     df_flowers.loc[i, "petal.length"] = norm(loc=3, scale=5).rvs()
#     df_flowers.loc[i, "petal.width"] = norm(loc=2, scale=2).rvs()
# 
# # customer uses your micro service to determine the varieties
# df_flowers["variety"] = micro_service_classify_iris(df_flowers)
# print(df_flowers)

In [ ]:
# # Let's assume that our id column is the index of the dataframe
# output = pd.DataFrame(df_flowers.variety)
# output['id'] = df_flowers.index
# output = output.rename(columns={'variety': 'prediction'})
# output = output.reindex(columns=["id", "prediction"])
# output.to_csv('iris_prediction.csv', index=False)

In [99]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25756 entries, 0 to 25755
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   AuthorId                    25756 non-null  object  
 1   Diet                        25756 non-null  category
 2   Age                         25756 non-null  int64   
 3   RecipeId                    25756 non-null  int64   
 4   Rating                      25756 non-null  float64 
 5   Like                        25756 non-null  object  
 6   TestSetId                   0 non-null      float64 
 7   Time                        25756 non-null  float64 
 8   HighCalories                25756 non-null  float64 
 9   HighProtein                 25756 non-null  object  
 10  LowFat                      25756 non-null  int64   
 11  LowSugar                    25756 non-null  object  
 12  HighFiber                   25756 non-null  int64   
 13  RecipeName      

OUR CODE 

In [120]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score

# Load your data
df = df_balanced

# Define features (X) and target variable (y)
features = ['Diet', 'Age', 'Rating', 'Time', 'CookTime', 'FiberContent', 'ProteinContent', 'SugarContent', 'PrepTime', 'LowSugar', 'LowFat', 'CarbohydrateContent', 'SodiumContent', 'CholesterolContent', 'SaturatedFatContent', 'FatContent', 'Calories', 'HighFiber', 'RecipeCategory']
target = 'Like'

# Drop rows with NaN values in the target column ('Like') and create a copy for training set
train_data = df.dropna(subset=[target]).copy()

# Convert 'Like' column to boolean (if not already)
train_data['Like'] = train_data['Like'].astype(bool)

# Split the data into features (X) and target variable (y)
X = train_data[features]
y = train_data['Like']

# Handle categorical variables using one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(
    n_estimators=100,  # You can adjust this
    random_state=42,
    min_samples_split=2,  # You can adjust this
    min_samples_leaf=1,  # You can adjust this
    max_features='log2'
)

# Fit the model on the training set
model.fit(X_train, y_train)

# Evaluate the model on the training set
y_train_pred = model.predict(X_train)

# Compute balanced accuracy for the training set
balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")

# Predict the 'Like' column for the testing set
y_test_pred = model.predict(X_test)

# Compute balanced accuracy for the testing set
balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)
print(f"Balanced Accuracy on Testing Set: {balanced_accuracy_test:.4f}")

Balanced Accuracy on Training Set: 0.9982
Balanced Accuracy on Testing Set: 0.7091


In [101]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
precision_train = precision_score(y_train, y_train_pred)
recall_train = recall_score(y_train, y_train_pred)
f1_train = f1_score(y_train, y_train_pred)
conf_matrix_train = confusion_matrix(y_train, y_train_pred)

# Print metrics for the training set
print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")
print(f"Precision on Training Set: {precision_train:.4f}")
print(f"Recall on Training Set: {recall_train:.4f}")
print(f"F1 Score on Training Set: {f1_train:.4f}")
print(f"Confusion Matrix on Training Set:\n{conf_matrix_train}")

# Predict the 'Like' column for the testing set
y_test_pred = model.predict(X_test)

# Compute metrics for the testing set
balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)

# Print metrics for the testing set
print(f"Balanced Accuracy on Testing Set: {balanced_accuracy_test:.4f}")
print(f"Precision on Testing Set: {precision_test:.4f}")
print(f"Recall on Testing Set: {recall_test:.4f}")
print(f"F1 Score on Testing Set: {f1_test:.4f}")
print(f"Confusion Matrix on Testing Set:\n{conf_matrix_test}")

Balanced Accuracy on Training Set: 0.9982
Precision on Training Set: 0.9980
Recall on Training Set: 0.9983
F1 Score on Training Set: 0.9982
Confusion Matrix on Training Set:
[[10298    21]
 [   17 10268]]
Balanced Accuracy on Testing Set: 0.7098
Precision on Testing Set: 0.7115
Recall on Testing Set: 0.7123
F1 Score on Testing Set: 0.7119
Confusion Matrix on Testing Set:
[[1810  749]
 [ 746 1847]]


In [114]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler

# Load your data
df = df_balanced

# Define features (X) and target variable (y)
features = ['Diet', 'Age', 'Rating', 'Time', 'CookTime', 'FiberContent', 'ProteinContent', 'SugarContent', 'PrepTime', 'LowSugar', 'LowFat', 'CarbohydrateContent', 'SodiumContent', 'CholesterolContent', 'SaturatedFatContent', 'FatContent', 'Calories', 'HighFiber']
target = 'Like'

# Drop rows with NaN values in the target column ('Like') and create a copy for the training set
train_data = df.dropna(subset=[target]).copy()

# Convert 'Like' column to boolean (if not already)
train_data['Like'] = train_data['Like'].astype(bool)

# Split the data into features (X) and target variable (y)
X = train_data[features]
y = train_data['Like']

# Handle categorical variables using one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the Logistic Regression model
# Increase max_iter and use the 'lbfgs' solver
model = LogisticRegression(
    random_state=42,
    solver='lbfgs',  # You can adjust this
    max_iter=1000,    # You can adjust this
    C=1.0,            # You can adjust this
    penalty='l2',     # You can adjust this
    class_weight=None  # You can adjust this
)


# Fit the model on the scaled training set
model.fit(X_train_scaled, y_train)

# Evaluate the model on the scaled training set
y_train_pred = model.predict(X_train_scaled)

# Compute balanced accuracy for the scaled training set
balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")

# Predict the 'Like' column for the scaled testing set
y_test_pred = model.predict(X_test_scaled)

# Compute balanced accuracy for the scaled testing set
balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)
print(f"Balanced Accuracy on Testing Set: {balanced_accuracy_test:.4f}")


Balanced Accuracy on Training Set: 0.6850
Balanced Accuracy on Testing Set: 0.6697


In [115]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler

# Load your data
df = df_balanced

# Define features (X) and target variable (y)
features = ['Diet', 'Age', 'Rating', 'Time', 'CookTime', 'FiberContent', 'ProteinContent', 'SugarContent', 'PrepTime', 'LowSugar', 'LowFat', 'CarbohydrateContent', 'SodiumContent', 'CholesterolContent', 'SaturatedFatContent', 'FatContent', 'Calories', 'HighFiber']
target = 'Like'

# Drop rows with NaN values in the target column ('Like') and create a copy for training set
train_data = df.dropna(subset=[target]).copy()

# Convert 'Like' column to boolean (if not already)
train_data['Like'] = train_data['Like'].astype(bool)

# Split the data into features (X) and target variable (y)
X = train_data[features]
y = train_data['Like']

# Handle categorical variables using one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the Neural Network model
model = MLPClassifier(
    hidden_layer_sizes=(100,),  # You can adjust this
    max_iter=500,                 # You can adjust this
    random_state=42,
    activation='relu',           # You can adjust this
    alpha=0.0001,                 # You can adjust this
    solver='adam',               # You can adjust this
    learning_rate='constant'      # You can adjust this
)


# Fit the model on the scaled training set
model.fit(X_train_scaled, y_train)

# Evaluate the model on the scaled training set
y_train_pred = model.predict(X_train_scaled)

# Compute balanced accuracy for the scaled training set
balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")

# Predict the 'Like' column for the scaled testing set
y_test_pred = model.predict(X_test_scaled)

# Compute balanced accuracy for the scaled testing set
balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)
print(f"Balanced Accuracy on Testing Set: {balanced_accuracy_test:.4f}")


Balanced Accuracy on Training Set: 0.7381
Balanced Accuracy on Testing Set: 0.7105


In [116]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import balanced_accuracy_score

# Load your data
df = df_balanced

# Define features (X) and target variable (y)
features = ['Diet', 'Age', 'Rating', 'Time', 'CookTime', 'FiberContent', 'ProteinContent', 'SugarContent', 'PrepTime', 'LowSugar', 'LowFat', 'CarbohydrateContent', 'SodiumContent', 'CholesterolContent', 'SaturatedFatContent', 'FatContent', 'Calories', 'HighFiber']
target = 'Like'

# Drop rows with NaN values in the target column ('Like') and create a copy for training set
train_data = df.dropna(subset=[target]).copy()

# Convert 'Like' column to boolean (if not already)
train_data['Like'] = train_data['Like'].astype(bool)

# Split the data into features (X) and target variable (y)
X = train_data[features]
y = train_data['Like']

# Handle categorical variables using one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Gradient Boosting model
model = GradientBoostingClassifier(
    n_estimators=100,     # You can adjust this
    random_state=42,
    learning_rate=0.1,    # You can adjust this
    max_depth=3,          # You can adjust this
    min_samples_split=2,  # You can adjust this
    min_samples_leaf=1,   # You can adjust this
    subsample=1.0         # You can adjust this
)


# Fit the model on the training set
model.fit(X_train, y_train)

# Evaluate the model on the training set
y_train_pred = model.predict(X_train)

# Compute balanced accuracy for the training set
balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")

# Predict the 'Like' column for the testing set
y_test_pred = model.predict(X_test)

# Compute balanced accuracy for the testing set
balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)
print(f"Balanced Accuracy on Testing Set: {balanced_accuracy_test:.4f}")

Balanced Accuracy on Training Set: 0.7392
Balanced Accuracy on Testing Set: 0.7123


In [118]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score

# Load your data
df = df_balanced

# Define features (X) and target variable (y)
features = ['Diet', 'Age', 'Rating', 'Time', 'CookTime', 'FiberContent', 'ProteinContent', 'SugarContent', 'PrepTime', 'LowSugar', 'LowFat', 'CarbohydrateContent', 'SodiumContent', 'CholesterolContent', 'SaturatedFatContent', 'FatContent', 'Calories', 'HighFiber', 'RecipeCategory']
target = 'Like'

# Drop rows with NaN values in the target column ('Like') and create a copy for training set
train_data = df.dropna(subset=[target]).copy()

# Convert 'Like' column to boolean (if not already)
train_data['Like'] = train_data['Like'].astype(bool)

# Split the data into features (X) and target variable (y)
X = train_data[features]
y = train_data['Like']

# Handle categorical variables using one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Support Vector Machine (SVM) model with adjusted parameters
model = SVC(
    C=1.0,            # You can adjust this
    kernel='rbf',     # You can adjust this
    degree=3,         # You can adjust this
    gamma='scale',    # You can adjust this
    random_state=42
)

# Fit the model on the training set
model.fit(X_train, y_train)

# Evaluate the model on the training set
y_train_pred = model.predict(X_train)

# Compute balanced accuracy for the training set
balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")

# Predict the 'Like' column for the testing set
y_test_pred = model.predict(X_test)

# Compute balanced accuracy for the testing set
balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)
print(f"Balanced Accuracy on Testing Set: {balanced_accuracy_test:.4f}")


KeyboardInterrupt: 

In [128]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import balanced_accuracy_score

# Load your data
df = df_balanced

# Define features (X) and target variable (y)
features = ['Diet', 'Age', 'Rating', 'Time', 'CookTime', 'FiberContent', 'ProteinContent', 'SugarContent', 'PrepTime', 'LowSugar', 'LowFat', 'CarbohydrateContent', 'SodiumContent', 'CholesterolContent', 'SaturatedFatContent', 'FatContent', 'Calories', 'HighFiber', 'RecipeCategory']
target = 'Like'

# Drop rows with NaN values in the target column ('Like') and create a copy for training set
train_data = df.dropna(subset=[target]).copy()

# Convert 'Like' column to boolean (if not already)
train_data['Like'] = train_data['Like'].astype(bool)

# Split the data into features (X) and target variable (y)
X = train_data[features]
y = train_data['Like']

# Handle categorical variables using one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the individual models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=20)
svm_model = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', random_state=42)

# Initialize the StackingClassifier with the individual models
stacking_model = StackingClassifier(
    estimators=[('rf', rf_model), ('svm', svm_model)],
    final_estimator=RandomForestClassifier(n_estimators=50, random_state=42)
)

# Fit the StackingClassifier on the training set
stacking_model.fit(X_train, y_train)

# Evaluate the StackingClassifier on the training set
y_train_pred = stacking_model.predict(X_train)
balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")

# Predict the 'Like' column for the testing set
y_test_pred = stacking_model.predict(X_test)

# Evaluate the StackingClassifier on the testing set
balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)
print(f"Balanced Accuracy on Testing Set: {balanced_accuracy_test:.4f}")


Balanced Accuracy on Training Set: 0.9329
Balanced Accuracy on Testing Set: 0.6737


In [119]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import balanced_accuracy_score

# Load your data
df = df_balanced

# Define features (X) and target variable (y)
features = ['Diet', 'Age', 'Rating', 'Time', 'CookTime', 'FiberContent', 'ProteinContent', 'SugarContent', 'PrepTime', 'LowSugar', 'LowFat', 'CarbohydrateContent', 'SodiumContent', 'CholesterolContent', 'SaturatedFatContent', 'FatContent', 'Calories', 'HighFiber', 'RecipeCategory']
target = 'Like'

# Drop rows with NaN values in the target column ('Like') and create a copy for training set
train_data = df.dropna(subset=[target]).copy()

# Convert 'Like' column to boolean (if not already)
train_data['Like'] = train_data['Like'].astype(bool)

# Split the data into features (X) and target variable (y)
X = train_data[features]
y = train_data['Like']

# Handle categorical variables using one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the individual models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=20)
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42, learning_rate=0.1, max_depth=3)

# Initialize the StackingClassifier with the individual models
stacking_model = StackingClassifier(
    estimators=[('rf', rf_model), ('gb', gb_model)],
    final_estimator=RandomForestClassifier(n_estimators=50, random_state=42)
)

# Fit the StackingClassifier on the training set
stacking_model.fit(X_train, y_train)

# Evaluate the StackingClassifier on the training set
y_train_pred = stacking_model.predict(X_train)
balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")

# Predict the 'Like' column for the testing set
y_test_pred = stacking_model.predict(X_test)

# Evaluate the StackingClassifier on the testing set
balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)
print(f"Balanced Accuracy on Testing Set: {balanced_accuracy_test:.4f}")

Balanced Accuracy on Training Set: 0.7866
Balanced Accuracy on Testing Set: 0.6787


In [124]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.feature_selection import SelectFromModel

# Load your data
df = df_balanced

# Define features (X) and target variable (y)
features = ['Diet', 'Age', 'Rating', 'Time', 'CookTime', 'FiberContent', 'ProteinContent', 'SugarContent', 'PrepTime', 'LowSugar', 'LowFat', 'CarbohydrateContent', 'SodiumContent', 'CholesterolContent', 'SaturatedFatContent', 'FatContent', 'Calories', 'HighFiber']
target = 'Like'

# Drop rows with NaN values in the target column ('Like') and create a copy for training set
train_data = df.dropna(subset=[target]).copy()

# Convert 'Like' column to boolean (if not already)
train_data['Like'] = train_data['Like'].astype(bool)

# Split the data into features (X) and target variable (y)
X = train_data[features]
y = train_data['Like']

# Handle categorical variables using one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest model with hyperparameter tuning
model = RandomForestClassifier(
    n_estimators=150,
    random_state=42,
    max_depth=10,  # Limiting the depth of the tree to make the model simpler
    min_samples_split=5,  # Minimum number of samples required to split an internal node
    min_samples_leaf=4,  # Minimum number of samples required to be at a leaf node
    max_features='sqrt'  # Number of features to consider when looking for the best split
)

# Optional: Implement GridSearchCV for further hyperparameter tuning
# parameter_grid = {
#     'n_estimators': [100, 200],
#     'max_depth': [10, 20, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['sqrt', 'log2']
# }
# grid_search = GridSearchCV(model, parameter_grid, cv=5, scoring='balanced_accuracy', n_jobs=-1)
# grid_search.fit(X_train, y_train)
# model = grid_search.best_estimator_

# Fit the model on the training set
model.fit(X_train, y_train)

# Evaluate the model on the training set
y_train_pred = model.predict(X_train)

# Compute metrics for the training set
balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
precision_train = precision_score(y_train, y_train_pred)
recall_train = recall_score(y_train, y_train_pred)
f1_train = f1_score(y_train, y_train_pred)
conf_matrix_train = confusion_matrix(y_train, y_train_pred)

# Print metrics for the training set
print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")
print(f"Precision on Training Set: {precision_train:.4f}")
print(f"Recall on Training Set: {recall_train:.4f}")
print(f"F1 Score on Training Set: {f1_train:.4f}")
print(f"Confusion Matrix on Training Set:\n{conf_matrix_train}")

# Predict the 'Like' column for the testing set
y_test_pred = model.predict(X_test)

# Compute metrics for the testing set
balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)

# Print metrics for the testing set
print(f"Balanced Accuracy on Testing Set: {balanced_accuracy_test:.4f}")
print(f"Precision on Testing Set: {precision_test:.4f}")
print(f"Recall on Testing Set: {recall_test:.4f}")
print(f"F1 Score on Testing Set: {f1_test:.4f}")
print(f"Confusion Matrix on Testing Set:\n{conf_matrix_test}")

# Optional: Print best parameters found by GridSearchCV
# print("Best parameters found by GridSearchCV:")
# print(grid_search.best_params_)

Balanced Accuracy on Training Set: 0.7876
Precision on Training Set: 0.7664
Recall on Training Set: 0.8263
F1 Score on Training Set: 0.7952
Confusion Matrix on Training Set:
[[7728 2591]
 [1786 8499]]
Balanced Accuracy on Testing Set: 0.7119
Precision on Testing Set: 0.7026
Recall on Testing Set: 0.7424
F1 Score on Testing Set: 0.7219
Confusion Matrix on Testing Set:
[[1744  815]
 [ 668 1925]]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.metrics import balanced_accuracy_score

# Load your data
df = df_balanced

# Define features (X) and target variable (y)
features = ['Diet', 'Age', 'Rating', 'Time', 'CookTime', 'FiberContent', 'ProteinContent', 'SugarContent', 'PrepTime', 'LowSugar', 'LowFat', 'CarbohydrateContent', 'SodiumContent', 'CholesterolContent', 'SaturatedFatContent', 'FatContent', 'Calories', 'HighFiber', 'RecipeCategory']
target = 'Like'

# Drop rows with NaN values in the target column ('Like') and create a copy for training set
train_data = df.dropna(subset=[target]).copy()

# Convert 'Like' column to boolean (if not already)
train_data['Like'] = train_data['Like'].astype(bool)

# Split the data into features (X) and target variable (y)
X = train_data[features]
y = train_data['Like']

# Handle categorical variables using one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the base model (Random Forest)
base_model = GradientBoostingClassifier(
    n_estimators=100,     # You can adjust this
    random_state=42,
    learning_rate=0.1,    # You can adjust this
    max_depth=3,          # You can adjust this
    min_samples_split=2,  # You can adjust this
    min_samples_leaf=1,   # You can adjust this
    subsample=1.0         # You can adjust this
)

# Initialize the BaggingClassifier
bagging_model = BaggingClassifier(
    base_model,
    n_estimators=50,  # You can adjust this
    max_samples=1.0,  # You can adjust this
    max_features=1.0,  # You can adjust this
    random_state=42
)

# Fit the BaggingClassifier on the training set
bagging_model.fit(X_train, y_train)

# Evaluate the BaggingClassifier on the training set
y_train_pred = bagging_model.predict(X_train)
balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")

# Predict the 'Like' column for the testing set
y_test_pred = bagging_model.predict(X_test)

# Evaluate the BaggingClassifier on the testing set
balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)
print(f"Balanced Accuracy on Testing Set: {balanced_accuracy_test:.4f}")


In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from xgboost import XGBClassifier
# from sklearn.metrics import balanced_accuracy_score
# 
# # Load your data
# df = pd.read_csv('final_data.csv')
# 
# # Define features (X) and target variable (y)
# features = ['Diet', 'Age', 'Rating', 'Time', 'HighCalories', 'HighProtein', 'LowFat', 'LowSugar',
#             'HighFiber', 'CookTime', 'PrepTime', 'RecipeCategory']
# target = 'Like'
# 
# # Drop rows with NaN values in the target column ('Like') for training set
# train_data = df.dropna(subset=[target]).copy()  # Create a copy to avoid the warning
# 
# # Use bootstrapping to split the training data
# # Assuming 'Like' column is boolean (True/False)
# train_data['Like'] = train_data['Like'].astype(bool)
# 
# # Split the data into features (X) and target variable (y) for training set
# X_train = pd.get_dummies(train_data[features])
# y_train = train_data['Like']
# 
# # Use bootstrapping to create a balanced sample
# # This is just an example, you may want to adjust the sample size based on your needs
# positive_samples = train_data[train_data['Like'] == True]
# negative_samples = train_data[train_data['Like'] == False]
# num_samples = min(len(positive_samples), len(negative_samples))
# balanced_sample = pd.concat([positive_samples.sample(num_samples), negative_samples.sample(num_samples)])
# 
# # Split the balanced sample into features (X) and target variable (y) for training set
# X_balanced = pd.get_dummies(balanced_sample[features])
# y_balanced = balanced_sample['Like']
# 
# # Initialize the XGBoost model
# model = XGBClassifier()
# 
# # Fit the model on the balanced sample
# model.fit(X_balanced, y_balanced)
# 
# # Predictions on the original training set
# y_train_pred = model.predict(X_train)
# 
# # Compute balanced accuracy for the training set
# balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
# 
# print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")


In [ ]:
# # Assuming you have already trained the model and made predictions on the test set
# # (Make sure that the 'Like' column in the test set is still NaN)
# 
# # Predict the 'Like' column for the testing set
# X_test = pd.get_dummies(df[df[target].isnull()][features])
# # Predict the 'Like' column for the testing set
# predictions = model.predict(X_test)
# 
# # Filter out rows where the true labels are known (not NaN)
# known_labels_mask = ~df[target].isnull()
# 
# # Filter out the true labels for known labels
# true_labels = df.loc[known_labels_mask, target]
# print(true_labels)
# # Convert true_labels to boolean (if not already)
# true_labels = true_labels.astype(bool)
# 
# # Filter out NaN values from the predictions
# predictions = predictions[:len(true_labels)]  # Ensure predictions and true_labels have the same length
# print(predictions)
# # Compute balanced accuracy for the testing set
# balanced_accuracy_test = balanced_accuracy_score(true_labels, predictions)
# 
# print(f"Balanced Accuracy on Testing Set: {balanced_accuracy_test:.4f}")


In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import balanced_accuracy_score
# 
# # Load your data
# df = pd.read_csv('final_data.csv')
# 
# # Define features (X) and target variable (y)
# features = ['Diet', 'Age', 'Rating', 'Time', 'HighCalories', 'HighProtein', 'LowFat', 'LowSugar',
#             'HighFiber', 'CookTime', 'PrepTime', 'RecipeCategory']
# target = 'Like'
# 
# # Drop rows with NaN values in the target column ('Like') for training set
# train_data = df.dropna(subset=[target]).copy()  # Create a copy to avoid the warning
# # Convert 'Like' column to boolean (if not already)
# train_data['Like'] = train_data['Like'].astype(bool)
# 
# # Split the data into features (X) and target variable (y) for training set
# X_train = pd.get_dummies(train_data[features])
# y_train = train_data['Like']
# 
# # Use bootstrapping to create a balanced sample
# # This is just an example, you may want to adjust the sample size based on your needs
# positive_samples = train_data[train_data['Like'] == True]
# negative_samples = train_data[train_data['Like'] == False]
# num_samples = min(len(positive_samples), len(negative_samples))
# balanced_sample = pd.concat([positive_samples.sample(num_samples), negative_samples.sample(num_samples)])
# 
# # Split the balanced sample into features (X) and target variable (y) for training set
# X_balanced = pd.get_dummies(balanced_sample[features])
# y_balanced = balanced_sample['Like']
# 
# # Initialize the Logistic Regression model
# model = LogisticRegression()
# 
# # Fit the model on the balanced sample
# model.fit(X_balanced, y_balanced)
# 
# # Predictions on the original training set
# y_train_pred = model.predict(X_train)
# 
# # Compute balanced accuracy for the training set
# balanced_accuracy_train = balanced_accuracy_score(y_train, y_train_pred)
# print(f"Balanced Accuracy on Training Set: {balanced_accuracy_train:.4f}")
